In [ ]:
from sklearn.model_selection import train_test_split
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm, ensemble
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

store = pd.read_csv('superstore_data.csv')
store.head()

In [ ]:
# analiza danych statystycznych
print(store.describe())
print(store.info())
print(store.isnull().values.any())

In [ ]:
store["Age"]= 2022 - store["Year_Birth"]
store.Age[:10]

In [ ]:
#rozkład wieku
store.Age.hist()

In [ ]:
#rozkład wieku wraz z odpowiedzią na kampanię
pd.crosstab(store.Age, store.Response).T

In [ ]:
#sprawdzenie wszystkich kolumn
store.columns

In [ ]:
# rozmiar
store.shape

In [ ]:
# sprawdzenie brakujących danych
store.isna().sum()

In [ ]:
#wypełnienie brakujących danych w Income wartością 0

store.Income = store.Income.fillna(0)

In [ ]:
# wyznaczenie macierzy korelacji

correlations = store.corr()
fig, ax = plt.subplots(figsize=(18, 18))
colormap = sns.color_palette("BrBG", 10)
ax= sns.heatmap(correlations, annot= True, linewidths= 2, cmap=colormap)
plt.show()

In [ ]:
# usuwanie nieprzydatnych danych
store.drop(['Id', 'Complain', 'Response', 'Year_Birth', 'NumWebVisitsMonth' ,'Dt_Customer'],
           axis=1, inplace = True)

In [ ]:
mapsMaritalStatus = {
    'Single': 0,
    'Married': 1,
    'Together': 2,
    'Divorced': 3,
    'Widow': 4,
    'YOLO': 5,
    'Alone': 6,
    'Absurd': 7
}
store['Marital_Status'] = store['Marital_Status'].apply(lambda x: mapsMaritalStatus[x])

In [ ]:
wyksztalcenie = {
    'Graduation': 0,
    'PhD': 1,
    '2n Cycle': 2,
    'Master': 3,
    'Basic': 4
}
store['Education'] = store['Education'].apply(lambda x: wyksztalcenie[x])

In [ ]:
# wyznaczenie macierzy korelacji

correlations = store.corr()
fig, ax = plt.subplots(figsize=(18, 18))
colormap = sns.color_palette("BrBG", 10)
ax= sns.heatmap(correlations, annot= True, linewidths= 2, cmap=colormap)
plt.show()

In [ ]:
#2. usuwanie nieprzydatnych danych
store.drop(['Education', 'Marital_Status', 'Recency', 'Teenhome', 'NumDealsPurchases', 'Age', 'MntGoldProds', 'Kidhome'],
           axis=1, inplace = True)

In [ ]:
# wyznaczenie macierzy korelacji

correlations = store.corr()
fig, ax = plt.subplots(figsize=(18, 18))
colormap = sns.color_palette("BrBG", 10)
ax= sns.heatmap(correlations, annot= True, linewidths= 2, cmap=colormap)
plt.show()

In [ ]:
# zbiór danych
X = store.drop('NumCatalogPurchases', axis=1).to_numpy()
X

In [ ]:
# zbiór etykiet
y = store.loc[:, 'NumCatalogPurchases'].to_numpy()
y

In [ ]:
# podział zbioru na dane treningowe i testowe
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=12345) 

In [ ]:
# uniwersalna metoda do trenowania i oceny modeli

def train_model(classifier, feature_vector_train, label, feature_vector_valid):
    # trenuj model
    classifier.fit(feature_vector_train, label)
    
    # wygeneruj przewidywania modelu dla zbioru testowego
    predictions = classifier.predict(feature_vector_valid)
    
    # dokonaj ewaluacji modelu na podstawie danych testowych
    scores = list(metrics.precision_recall_fscore_support(predictions, y_test,zero_division = 1))
    score_vals = [
        scores[0][0],
        scores[1][0],
        scores[2][0]
    ]
    score_vals.append(metrics.accuracy_score(predictions, y_test))
    return score_vals

In [ ]:
# MODEL 1 - regresja logistyczna 
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

classifier = linear_model.LogisticRegression(max_iter=1000)

accuracy = train_model(classifier, X_train, y_train, X_test)
accuracy_compare = {'LR': accuracy}
print ("LR: ", accuracy)

In [ ]:
# MODEL 2 - Support Vector Machine
accuracy = train_model(svm.SVC(), X_train, y_train, X_test)
accuracy_compare['SVM'] = accuracy
print ("SVM" , accuracy)

In [ ]:
# MODEL 3 - Random Forest Tree 
accuracy = train_model(ensemble.RandomForestClassifier(), X_train, y_train, X_test)
accuracy_compare['RF'] = accuracy
print ("RF: ", accuracy)

In [ ]:
# porównanie modeli

df_compare = pd.DataFrame(accuracy_compare, index = ['precision', 'recall', 'f1 score', 'accuracy'])
df_compare.plot(kind='bar')

In [ ]:
# działania korygujące - zastosowanie sieci neuronowej

# MODEL 4 - neural network
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(hidden_layer_sizes=(10, 10, 10), max_iter=1000)
accuracy = train_model(mlp, X_train, y_train, X_test)
accuracy_compare['neural network'] = accuracy
print ("neural network" , accuracy)

In [ ]:
# działania korygujące - hiperparametry

# MODEL 5 - Support Vector Machine
accuracy = train_model(svm.SVC(gamma='auto'), X_train, y_train, X_test)
accuracy_compare['SVM'] = accuracy
print ("SVM gamma='auto'" , accuracy)

# MODEL 6 - Support Vector Machine
accuracy = train_model(svm.SVC(kernel='sigmoid'), X_train, y_train, X_test)
accuracy_compare['SVM'] = accuracy
print ("SVM kernel='sigmoid'" , accuracy)

# MODEL 7 - Support Vector Machine
accuracy = train_model(svm.SVC(degree=4), X_train, y_train, X_test)
accuracy_compare['SVM'] = accuracy
print ("SVM degree=4" , accuracy)